In [6]:
import os
import nibabel as nib
import numpy as np
from PIL import Image
import tqdm



In [7]:
import m_yolo
from m_yolo import nii_rgb


file = os.listdir('dataset_nii/train/images')
images = 'dataset_nii/train/images'
masks = 'dataset_nii/train/labels'
output_dir = 'dataset/train'
os.makedirs(output_dir, exist_ok=True)

for i in tqdm.tqdm(range(len(file))):
    image_path = os.path.join(images, file[i])
    mask_path = os.path.join(masks, file[i])
    nii_rgb.save_slices_as_rgb(image_path, mask_path, output_dir, name=file[i], angle=0)


  0%|          | 3/1082 [00:03<23:55,  1.33s/it]


KeyboardInterrupt: 

images: , 44792
labels: , 44792


In [58]:
# read the files from image and label directory and make sure that their size is the same
import matplotlib.pyplot as plt 
import os
files = os.listdir('dataset/train/images')
print('images: ,', len(files))
files = os.listdir('dataset/train/masks')
print('labels: ,', len(files))

unique_values = []
for i in tqdm.tqdm(range(len(files))):
    image_name = 'dataset/train/images/' + files[i]
    mask_name = 'dataset/train/masks/' + files[i]
    mask = plt.imread(mask_name)
    uniq = np.unique(mask)
    unique_values.append(len(uniq))
    img = plt.imread(image_name)


    if plt.imread(image_name)[:,:,0].shape  == plt.imread(mask_name).shape:
        continue    
    else:
        print(image_name), print(plt.imread(image_name).shape), print(image_name)

images: , 278190
labels: , 278190


100%|██████████| 278190/278190 [12:39<00:00, 366.35it/s]


In [60]:
'total number of trainig masks: ', np.sum(unique_values)

('total number of trainig masks: ', 5332982)

In [4]:
import os 
len(os.listdir('dataset/train/images')), len(os.listdir('dataset/train/labels'))

(278190, 278190)

In [64]:
import cv2
import numpy as np
from PIL import Image
def convert_mask_to_segmentation_(mask_path, output_txt_path, class_mapping):
    # Load the mask image
    mask_image = Image.open(mask_path)
    
    # Ensure the mask is in integer mode
    if mask_image.mode not in ['L', 'I']:
        mask_image = mask_image.convert('I')
    
    # Convert the mask image to a numpy array
    mask_np = np.array(mask_image).astype(np.int32)
    
    # Identify unique labels (exclude 0 if it's background)
    unique_labels = np.unique(mask_np)
    unique_labels = unique_labels[unique_labels != 0]
    
    # Image dimensions for normalization
    height, width = mask_np.shape[:2]
    
    # Open the output file
    with open(output_txt_path, 'w') as f:
        for label in unique_labels:
            # Verify the label exists in class_mapping
            if label not in class_mapping:
                print(f"Warning: Label {label} not found in class mapping.")
                continue
            
            # Create a binary mask for the current label
            mask_bin = (mask_np == label).astype(np.uint8)
            contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            for contour in contours:
                if len(contour) < 3:
                    continue  # Skip if contour has less than 3 points

                # Normalize and flatten the contour coordinates
                contour = contour.reshape(-1, 2).astype(float)
                contour[:, 0] = contour[:, 0] / width  # Normalize x
                contour[:, 1] = contour[:, 1] / height  # Normalize y
                segmentation = contour.flatten().round(6).tolist()
                
                # Format segmentation points as strings
                segmentation_str = ' '.join(f"{coord:.6f}" for coord in segmentation)
                class_id = class_mapping[label]
                
                # Write to file in format: <class_id> <points>
                f.write(f"{class_id} {segmentation_str}\n")


class_mapping = {0: 1, 1: 2, 2: 3, 3: 4, 4:5, 5:6, 6:7, 7:8, 8:9, 9:10, 10:11, 11:12, 12:13, 13:14, 14:15, 15:16, 16:17, 17:18, 18:19, 19:20, 20:21, 21:22, 22:23, 23:24, 
                 24:25, 25:26, 26:27, 27:28, 28:29, 29:30, 30:31, 31:32, 32:33, 33:34, 34:35, 35:36, 36:37, 37:38, 38:39, 39:40, 40:41, 41:42, 42:43, 43:44, 44:45, 45:46, 
                 46:47, 47:48, 48:49, 49:50, 50:51, 51:52, 52:53, 53:54, 54:55, 55:56, 56:57, 57:58, 58:59, 59:60, 60:61, 61:62, 62:63, 63:64, 64:65, 65:66, 66:67, 67:68, 
                 68:69, 69:70, 70:71, 71:72, 72:73, 73:74, 74:75, 75:76, 76:77, 77:78, 78:79, 79:80, 80:81, 81:82, 82:83, 83:84, 84:85, 85:86, 86:87, 87:88, 88:89, 89:90, 90:91,
                   91:92, 92:93, 93:94, 94:95, 95:96, 96:97, 97:98, 98:99, 99:100, 100:101, 101:102, 102:103, 103:104, 104:105, 105:106, 106:107, 107:108, 108:109, 109:110, 110:111, 
                   111:112, 112:113, 113:114, 114:115, 115:116, 116:117, 117:118, 118:119, 119:120, 120:121, 121:122, 122:123, 123:124, 124:125, 125:126, 126:127, 127:128, 128:129, 
                   129:130, 130:131, 131:132, 132:133, 133:134, 134:135, 135:136, 136:137, 137:138, 138:139, 139:140, 140:141, 141:142, 142:143, 143:144, 144:145, 145:146, 146:147, 
                   147:148, 148:149, 149:150, 150:151, 151:152, 152:153, 153:154, 154:155, 155:156, 156:157, 157:158, 158:159, 159:160, 160:161, 161:162, 162:163, 163:164, 164:165, 
                   165:166, 166:167, 167:168, 168:169, 169:170, 170:171, 171:172, 172:173, 173:174, 174:175, 175:176, 176:177, 177:178, 178:179, 179:180, 180:181, 181:182, 182:183,
                     183:184, 184:185, 185:186, 186:187, 187:188, 188:189, 189:190, 190:191, 191:192, 192:193, 193:194, 194:195, 195:196, 196:197, 197:198, 198:199, 199:200, 200:201}



In [67]:
input_mask_path = 'dataset/train/masks/'
file = os.listdir(input_mask_path)
for i in tqdm.tqdm(range(len(file))):
    input_mask_path = 'dataset/train/masks/' + file[i]
    output_dir = 'dataset/train/labels/'
    os.makedirs(output_dir, exist_ok=True)
    output_txt_path = output_dir + file[i].replace('.tiff', '.txt')
    convert_mask_to_segmentation_(input_mask_path, output_txt_path, class_mapping)
    


100%|██████████| 278190/278190 [15:57<00:00, 290.45it/s]


In [68]:
input_mask_path = 'dataset/test/masks/'
file = os.listdir(input_mask_path)
for i in tqdm.tqdm(range(len(file))):
    input_mask_path = 'dataset/test/masks/' + file[i]
    output_dir = 'dataset/test/labels/'
    os.makedirs(output_dir, exist_ok=True)
    output_txt_path = output_dir + file[i].replace('.tiff', '.txt')
    convert_mask_to_segmentation_(input_mask_path, output_txt_path, class_mapping)
    


100%|██████████| 25578/25578 [02:08<00:00, 198.63it/s]
